In [10]:
import boto3
import logging
import subprocess
import os
import time
from io import StringIO
from botocore.exceptions import NoCredentialsError, ClientError

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

def key_pair_exists(ec2_client, key_name):
    """
    Check if a key pair with the given name already exists.

    Parameters:
    ec2_client (boto3.client): The EC2 client.
    key_name (str): The name of the key pair to check.

    Returns:
    bool: True if the key pair exists, False otherwise.
    """
    try:
        key_pairs = ec2_client.describe_key_pairs(KeyNames=[key_name])
        return bool(key_pairs['KeyPairs'])
    except ClientError as e:
        if e.response['Error']['Code'] == 'InvalidKeyPair.NotFound':
            return False
        else:
            raise

def create_key_pair(key_name, save_to_file_path):
    """
    Create a new EC2 key pair with the given name if it doesn't already exist,
    and save the private key to a file. Then, set the file's permissions to read-only
    for the file's owner.

    Parameters:
    key_name (str): The name of the key pair to create.
    save_to_file_path (str): The local file path where the private key should be saved.
    """
    ec2_client = boto3.client('ec2')
    if key_pair_exists(ec2_client, key_name):
        logger.info(f"Key pair '{key_name}' already exists in AWS.")
        return
    if os.path.exists(save_to_file_path):
        logger.error(f"File '{save_to_file_path}' already exists. Will not overwrite.")
        return
    
    try:
        # Create the key pair
        key_pair = ec2_client.create_key_pair(KeyName=key_name)
        
        # Save the private key to a file
        with open(save_to_file_path, 'w') as file:
            file.write(key_pair['KeyMaterial'])
        
        logger.info(f"Key pair '{key_name}' created and saved to '{save_to_file_path}'.")

        # Set the file's permissions to read-only for the file's owner
        subprocess.run(['chmod', '400', save_to_file_path], check=True)
        logger.info(f"Permissions for '{save_to_file_path}' set to 'read-only' for the file's owner.")
    except ClientError as e:
        logger.error(f"Failed to create key pair '{key_name}': {e}")
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to set permissions for '{save_to_file_path}': {e}")

def terminate_instance(instance_name):
    """
    Delete (terminate) an EC2 instance by its name tag.
    """
    ec2_client = boto3.client('ec2')
    instance_id = get_instance_id_by_name(ec2_client, instance_name)
    # check to see if the instance is stopped
    state,public_ip = get_instance_info(instance_name,key_path,user,instruct=False,meta=False)
    # if the instance is running, stop it
    if state == 'running':
        stop_instance(instance_name)
    if instance_id:
        try:
            logging.info(f"Removing SSH key for ip {public_ip} from known hosts...")
            subprocess.run(['ssh-keygen', '-R', public_ip], stderr=subprocess.DEVNULL, check=True)
            # Get all tags for the instance
            instance = ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]
            tags = instance.get('Tags', [])
            
            if tags:
                # Delete all tags from the instance
                ec2_client.delete_tags(Resources=[instance_id], Tags=tags)
                logger.info(f"All tags removed from the instance '{instance_name}' with ID {instance_id}.")
            
            ec2_client.terminate_instances(InstanceIds=[instance_id])
            logger.info(f"Instance '{instance_name}' with ID {instance_id} has been terminated.")
        except ClientError as e:
            logger.error(f"Error terminating instance {instance_name}: {e}")
    else:
        logger.info(f"No instance found with name '{instance_name}'.")

def get_instance_info(instance_name,key_path,user,instruct=True,meta=True):
    """
    Retrieve the running state and public IP address of an EC2 instance by its name tag.
    """
    ec2_client = boto3.client('ec2')
    try:
        response = ec2_client.describe_instances(
            Filters=[
                {'Name': 'tag:Name', 'Values': [instance_name]},
                {'Name': 'instance-state-name', 'Values': ['pending', 'running', 'shutting-down', 'terminated', 'stopping', 'stopped']}
            ]
        )
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                state = instance['State']['Name']
                public_ip = instance.get('PublicIpAddress', 'N/A')
                logger.info(f"Instance '{instance_name}' is in state: {state} with Public IP: {public_ip}")
                # get the available disk space on the instance
                if state == 'running' and meta:
                    try:
                        result = subprocess.run(['ssh', '-i', key_path, f'{user}@{public_ip}', 'df', '-h', '/'], capture_output=True, text=True, check=True)
                        logger.info(f"Disk space on instance '{instance_name}':\n{result.stdout}")
                    except subprocess.CalledProcessError as e:
                        logger.error(f"Error getting disk space on instance {instance_name}: {e}")
                 # Log the shell command for SSH access
                ssh_command = f"ssh -i {key_path} {user}@{public_ip}"
                if instruct:
                    logger.info(f"To SSH into the instance '{instance_name}', use the following command: {ssh_command}")
                return state, public_ip
        logger.info(f"No instance found with name '{instance_name}'.")
        return None, None
    except ClientError as e:
        logger.error(f"Error retrieving state and IP for instance {instance_name}: {e}")
        return None, None
    
def get_instance_id_by_name(ec2_client, instance_name):
    """
    Find the instance ID of the first EC2 instance with a given name tag.
    """
    try:
        response = ec2_client.describe_instances(
            Filters=[
                {'Name': 'tag:Name', 'Values': [instance_name]},
                {'Name': 'instance-state-name', 'Values': ['running', 'stopped']}
            ]
        )
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                return instance['InstanceId']
    except ClientError as e:
        logger.error(f"Error finding instance by name {instance_name}: {e}")
    return None

def reboot_instance(instance_name):

    """
    Start an EC2 instance by its name tag.
    """
    ec2_client = boto3.client('ec2')
    instance_id = get_instance_id_by_name(ec2_client, instance_name)
    if instance_id:
        try:
            response = ec2_client.reboot_instances(InstanceIds=[instance_id])
            logger.info(f"Rebooting instance '{instance_name}' with ID {instance_id}")
            return response
        except ClientError as e:
            logger.error(f"Error rebooting instance {instance_name}: {e}")
    else:
        logger.info(f"Instance named '{instance_name}' not found.")

def start_instance(instance_name):
    """
    Start an EC2 instance by its name tag.
    """
    ec2_client = boto3.client('ec2')
    instance_id = get_instance_id_by_name(ec2_client, instance_name)
    if instance_id:
        try:
            response = ec2_client.start_instances(InstanceIds=[instance_id])
            logger.info(f"Starting instance '{instance_name}' with ID {instance_id}")
            # Wait for the instance to start
            waiter = ec2_client.get_waiter('instance_running')
            waiter.wait(InstanceIds=[instance_id])
            logger.info(f"Instance '{instance_name}' with ID {instance_id} has started.")
            get_instance_info(instance_name,key_path,user)
        except ClientError as e:
            logger.error(f"Error starting instance {instance_name}: {e}")
    else:
        logger.info(f"Instance named '{instance_name}' not found.")

def stop_instance(instance_name):
    """
    Stop an EC2 instance by its name tag.
    """
    ec2_client = boto3.client('ec2')
    instance_id = get_instance_id_by_name(ec2_client, instance_name)
    if instance_id:
        try:
            response = ec2_client.stop_instances(InstanceIds=[instance_id])
            logger.info(f"Stopping instance '{instance_name}' with ID {instance_id}")
            # Wait for the instance to stop
            waiter = ec2_client.get_waiter('instance_stopped')
            waiter.wait(InstanceIds=[instance_id])
            logger.info(f"Instance '{instance_name}' with ID {instance_id} has stopped.")
        except ClientError as e:
            logger.error(f"Error stopping instance {instance_name}: {e}")
    else:
        logger.info(f"Instance named '{instance_name}' not found.")

def get_vpc_by_name(ec2_resource, vpc_name):
    try:
        for vpc in ec2_resource.vpcs.all():
            for tag in vpc.tags or []:
                if tag['Key'] == 'Name' and tag['Value'] == vpc_name:
                    return vpc
        return None
    except NoCredentialsError:
        logging.error("Credentials not found.")
        raise
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

def check_and_create_ec2_instance(ec2_resource, ec2_client, instance_name, vpc, key_pair_name,security_group_id):
    try:
        instances = ec2_client.describe_instances(Filters=[
            {'Name': 'tag:Name', 'Values': [instance_name]},
            {'Name': 'vpc-id', 'Values': [vpc.id]}
        ])
        if instances['Reservations']:
            logging.info(f"EC2 instance '{instance_name}' already exists in VPC '{vpc.id}'.")
            get_instance_info(instance_name,key_path,user)
            return
        else:
            # Create a new EC2 instance within the VPC
            instance = ec2_resource.create_instances(
                ImageId=image_id,  # Example AMI ID, replace with a current Ubuntu AMI
                MinCount=1,
                MaxCount=1,
                InstanceType='t2.micro', #c7a.medium $0.05132, t2.micro ~8G, t2.small $.023 ~8G, t2.medium $.031 +2G?, t3.medium ~same as t2.medium, m1.small
                KeyName=key_pair_name,
                NetworkInterfaces=[{'SubnetId': list(vpc.subnets.all())[0].id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True}],
                TagSpecifications=[{'ResourceType': 'instance', 'Tags': [{'Key': 'Name', 'Value': instance_name}]}]
            )[0]
            logging.info(f"EC2 instance '{instance_name}' created with ID: {instance.id}")
            try:
                response = ec2_client.modify_instance_attribute(InstanceId=instance.id,Groups=[security_group_id])
                logging.info(f"Security groups for instance {instance.id} have been updated.")
            except ClientError as e:
                logging.error(f"Error modifying security group for instance {instance_name}: {e}")
            # wait for the instance to enter the running state
            logging.info(f"Waiting for instance '{instance_name}' to enter the running state...")
            instance.wait_until_running()
            logging.info(f"Instance '{instance_name}' is running.")
            state,public_ip = get_instance_info(instance_name,key_path,user)
            
            # in a try block, attempt to ssh into the instance. If the instance is not ready, loop and retry every 2 seconds for a maximum of 1 minute until the instance is ready.
            ready = False
            start_time = time.time()
            added_to_known_hosts = False
            while not ready and time.time() - start_time < 60:
                try:
                    if not added_to_known_hosts:
                        # fetch the EC2 instance's public key and add it to known hosts file before running SSH commands. 
                        logging.info(f"Adding ip {public_ip} to known hosts...")
                        
                        # Run ssh-keyscan and capture the output
                        result = subprocess.run(['ssh-keyscan', '-H', public_ip], capture_output=True, text=True, check=True)
                        new_keys = result.stdout.splitlines()
                        # Add the keys to the known hosts file
                        with open(os.path.expanduser('~/.ssh/known_hosts'), 'a') as known_hosts_file:
                            known_hosts_file.write(result.stdout)

                        # Check each new key separately
                        with open(os.path.expanduser('~/.ssh/known_hosts'), 'r') as known_hosts_file:
                            known_hosts = known_hosts_file.read()
                        for new_key in new_keys:
                            if new_key in known_hosts:
                                logging.info(f"A key for {public_ip} has been added to the known hosts file.")
                                added_to_known_hosts = True
                            else:
                                logging.info(f"A key for {public_ip} has NOT been added to the known hosts file.")
                    logging.info(f"Attempting to SSH to instance {instance_name}...")
                    subprocess.run(['ssh', '-i', key_path, f'{user}@{public_ip}', 'true'], check=True)
                    logging.info(f"Connected via SSH to instance {instance_name}.")
                    ready = True
                except subprocess.CalledProcessError as e:
                    #logging.error(f"Error connecting to instance {instance_name}: {e}")
                    time.sleep(1)
            if ready:
                # run a command on the remote host to clone the os-configuration repo to the root directory, overwriting any existing configuration files like .bashrc
                logging.info(f"Cloning os-configuration repo...")
                subprocess.run(['ssh', '-i', key_path, f'{user}@{public_ip}', 'git', 'clone', 'https://github.com/jpmalek/os-configuration.git'],stderr=subprocess.DEVNULL,check=True) 
                # copy in the .bashrc file
                logging.info(f"Copying in .bashrc...")
                subprocess.run(['ssh', '-i', key_path, f'{user}@{public_ip}', 'cp', 'os-configuration/.bashrc','.'], check=True)
                setup_new_instance(instance_name)
            else:
                logging.error(f"Failed to SSH to instance {instance_name} after 1 minute.") 

    except NoCredentialsError:
        logging.error("Credentials not found.")
        raise
    except Exception as e:
        logging.error(f"An error occurred while creating EC2 instance: {e}")
        raise

def setup_new_instance(instance_name):
    """
    Run initialization scripts from os-configuration repo on a new EC2 instance.
    """
    logger.info(f"Setting up instance '{instance_name}'...")
    # Get the instance's public IP address
    state, public_ip = get_instance_info(instance_name,key_path,user,instruct=False)
    if state != 'running':
        logger.info(f"Instance '{instance_name}' is not running. Will not run setup scripts.")
        return
    try:
        # Run a remote command to clone the os-configuration repo
        subprocess.run(['ssh', '-i', key_path, f'{user}@{public_ip}', 'bash','os-configuration/init.sh' ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    except subprocess.CalledProcessError as e:
        logger.error(f"Error running setup scripts on instance {instance_name}: {e}")
    state, public_ip = get_instance_info(instance_name,key_path,user,instruct=False)

def create_security_group(vpc,name):
    try:
        security_group = list(ec2_resource.security_groups.filter(Filters=[{'Name': 'group-name', 'Values': [name]}]))
        if security_group:
            # If the list is not empty, the security group exists
            logging.info(f"Security group '{name}' exists.")
            return security_group[0]
    except ClientError as e:
        logger.error(f"Failed to discover security group for SSH: {e}")    
    
    try:
        # Create a security group
        sg = ec2_resource.create_security_group(
            GroupName=name, #'SSHAccess',
            Description='Security group for SSH access',
            VpcId=vpc.id
        )
        # Add an inbound rule to allow SSH (port 22) from any IP
        sg.authorize_ingress(
            IpPermissions=[
                {
                    'IpProtocol': 'tcp',
                    'FromPort': 22,
                    'ToPort': 22,
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }
            ]
        )
        logger.info(f"Security group '{sg.id}' created and configured for SSH access.")
        return sg
    except ClientError as e:
        logger.error(f"Failed to create or update security group for SSH (may already have been configured): {e}")

def copy_files_to_instance(instance_name,file_name):
    """
    Copy files to an EC2 instance using SCP.
    """
    state, public_ip = get_instance_info(instance_name,key_path,user)
    if state != 'running':
        logger.info(f"Instance '{instance_name}' is not running. Will not copy files.")
        return
    
    try:
        # Copy a file to the instance
        subprocess.run(['scp', '-i', key_path, file_name, f'{user}@{public_ip}:~/'], check=True)
        logger.info(f"File '{file_name}' copied to instance '{instance_name}' at {public_ip}:~/")
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to copy file to instance {instance_name}: {e}")

def create_and_attach_ebs_volume(instance_name, volume_size_gb, volume_type='gp2'):
    """
    Create an EBS volume and attach it to an EC2 instance.
    """
    # TODO: pick up here again when an EBS volume is needed. It's 99% ready to go.
    # ec2_client = boto3.client('ec2')
    # ec2_resource = boto3.resource('ec2')
    # state, public_ip = get_instance_info(instance_name,key_path,user)
    # if state != 'running':
    #     logger.info(f"Instance '{instance_name}' is not running. Will not create or attach volume.")
    #     return
    # try:
    #     # check to see if there's already an existing volume tagged with this instance name
    #     volumes = ec2_client.describe_volumes(Filters=[{'Name': 'tag:Name', 'Values': [instance_name]}])
    #     if volumes['Volumes']:
    #         logger.info(f"Volume already exists with name '{instance_name}'.")
    #         volume=volumes['Volumes'][0]
    #     else:
    #         # Create an EBS volume
    #         volume = ec2_resource.create_volume(
    #             AvailabilityZone=ec2_client.describe_instances(InstanceIds=[get_instance_id_by_name(ec2_client, instance_name)])['Reservations'][0]['Instances'][0]['Placement']['AvailabilityZone'],
    #             Size=volume_size_gb,
    #             VolumeType=volume_type
    #         )
    #         logger.info(f"EBS volume '{volume.id}' created with size {volume_size_gb} GB and type {volume_type}.")
        
    #     # Attach the volume to the instance
    #     response = ec2_client.attach_volume(
    #         Device='/dev/sdf',
    #         InstanceId=get_instance_id_by_name(ec2_client, instance_name),
    #         VolumeId=volume.id
    #     )
    #     logger.info(f"Volume '{volume.id}' attached to instance '{instance_name}' as device '/dev/sdf'.")
    # except ClientError as e:
    #     logger.error(f"Failed to create or attach EBS volume to instance {instance_name}: {e}")

#if __name__ == "__main__":
# Specify your AWS profile and region
aws_profile = 'default'
region_name = 'us-west-2'
image_id = 'ami-008fe2fc65df48dac' # Canonical, Ubuntu, 22.04 LTS, amd64 jammy image build on 2023-12-07, includes Python 3.10.12
    
# Load specified AWS profile
boto3.setup_default_session(profile_name=aws_profile, region_name=region_name)
    
# Initialize Boto3 EC2 resource and client
ec2_resource = boto3.resource('ec2')
ec2_client = boto3.client('ec2')

user = 'ubuntu'    
vpc_name = 'oregon'
instance_name = 'utility'
key_name = instance_name
security_group_name = 'SSHAccess'
key_path = os.path.expanduser(f'~/.aws/{key_name}.pem')

create_key_pair(key_name, key_path)

# Get the VPC by name
vpc = get_vpc_by_name(ec2_resource, vpc_name)
# Create a security group for SSH access
sg = create_security_group(vpc,security_group_name)

if vpc:
    logging.info(f"VPC '{vpc_name}' found with ID: {vpc.id}")
    # Check for an EC2 instance by name in the VPC and create it if it doesn't exist
    check_and_create_ec2_instance(ec2_resource, ec2_client, instance_name, vpc,key_name,sg.id)
else:
    logging.error(f"VPC '{vpc_name}' not found.")


2024-03-21 16:38:20,737 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2024-03-21 16:38:21,243 - INFO - Key pair 'utility' already exists in AWS.
2024-03-21 16:38:21,727 - INFO - Security group 'SSHAccess' exists.
2024-03-21 16:38:21,730 - INFO - VPC 'oregon' found with ID: vpc-e1c44684
2024-03-21 16:38:22,036 - INFO - EC2 instance 'utility' already exists in VPC 'vpc-e1c44684'.


In [8]:
terminate_instance(instance_name)

2024-03-21 16:24:16,701 - INFO - Instance 'utility' is in state: running with Public IP: 54.148.216.33
2024-03-21 16:24:18,134 - INFO - Disk space on instance 'utility':
Filesystem      Size  Used Avail Use% Mounted on
/dev/root       7.6G  7.5G   69M 100% /

2024-03-21 16:24:18,806 - INFO - Stopping instance 'utility' with ID i-0b056be13b4342211
2024-03-21 16:24:49,621 - INFO - Instance 'utility' with ID i-0b056be13b4342211 has stopped.
2024-03-21 16:24:49,623 - INFO - Removing SSH key for ip 54.148.216.33 from known hosts...


# Host 54.148.216.33 found: line 56
# Host 54.148.216.33 found: line 57
# Host 54.148.216.33 found: line 58
/Users/jeffmalek/.ssh/known_hosts updated.
Original contents retained as /Users/jeffmalek/.ssh/known_hosts.old


2024-03-21 16:24:49,990 - INFO - All tags removed from the instance 'utility' with ID i-0b056be13b4342211.
2024-03-21 16:24:50,236 - INFO - Instance 'utility' with ID i-0b056be13b4342211 has been terminated.


In [ ]:
state = get_instance_info(instance_name,key_path,user)
state 

In [ ]:
start_instance(instance_name)

In [ ]:
reboot_instance(instance_name)

In [ ]:
stop_instance(instance_name)

In [112]:
response = ec2_client.describe_instances(
            Filters=[
                {'Name': 'tag:Name', 'Values': ['utility']},
                {'Name': 'instance-state-name', 'Values': ['pending', 'running', 'shutting-down', 'terminated', 'stopping', 'stopped']}
            ]
        )

In [ ]:
#copy_files_to_instance(instance_name,'requirements.txt') 
#copy_files_to_instance(instance_name,'init.sh')
copy_files_to_instance(instance_name,'../lake-washington-and-sammamish-temps/Dockerfile')
#copy_files_to_instance(instance_name,'../lake-washington-and-sammamish-temps/compose.yaml')
#copy_files_to_instance(instance_name,'export_aws_credentials_to_env.sh')
#copy_files_to_instance(instance_name,'/Users/jeffmalek/.aws/credentials')